TensorFlowを利用したMNISTのトレーニングを、Kubeflow Pipelinesに実装してみる。

https://github.com/gowatana/demo-kubeflow/blob/main/tutorials/demo-tensorflow-mnist.ipynb

実行環境
* Kubeflow 1.6.1
* Notebook: kubeflownotebookswg/jupyter-tensorflow-full:v1.6.0
* NGC Image: nvcr.io/nvidia/tensorflow:23.03-tf2-py3

# 事前準備

## KubeflowでNotebookを作成する
* tensorflow, kfp などは、Notebookのコンテナ イメージ（jupyter-tensorflow-full:v1.6.0）に含まれている。

## PVCの準備
PVCを作成してあることを確認する。
PVCは、Kubeflow UIのVolumesメニューから作成しておく。

In [ ]:
!kubectl get pvc demo-vol-01

# パイプラインの作成（モデルの作成→トレーニング→保存）

kfpで、パイプラインのYAMLを作成する。
* これは、トレーニングとモデルの保存を実行するパイプライン

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp

def train_and_save_model():
    import tensorflow as tf
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)
    model.evaluate(x_test, y_test, verbose=2)

    model.save('/mnt/demo-vol-01/mnist_saved_model')

train_and_save_op = comp.func_to_container_op(train_and_save_model, base_image='tensorflow/tensorflow:latest')

@dsl.pipeline(
    name='Mnist Training Pipeline',
    description='A pipeline that trains an MNIST model and saves it to PVC.'
)

def mnist_pipeline():
    train_and_save_task = train_and_save_op()
    train_and_save_task.add_pvolumes({'/mnt/demo-vol-01': dsl.PipelineVolume(pvc="demo-vol-01")})

kfp.compiler.Compiler().compile(mnist_pipeline, 'mnist_pipeline.yaml')

コンパイルで生成されたYAMLを確認する。

In [ ]:
!cat mnist_pipeline.yaml

生成されたYAMLはローカルにダウンロードして、Kubeflow UIの「Pipelines (KFP)」からアップロードする。
* ブラウザで、Kubeflow UI を開く
* Pipelines 画面を開く
* 「Upload Pipeline」からアップロードする

パイプラインは、Argo Workflow（workflowリソース）として作成される。

In [ ]:
!kubectl get workflow

# パイプラインの作成（テスト データでの推論を追加）
kfpで、パイプラインのYAMLを作成する。
* トレーニングとモデルの保存（train_and_save_op） → 推論（predict_op）

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp
import tensorflow as tf
import numpy as np

# トレーニング関数は以前のまま
def train_and_save_model():
    import tensorflow as tf
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)
    model.evaluate(x_test, y_test, verbose=2)

    model.save('/mnt/demo-vol-01/mnist_saved_model')

# 推論を行う関数を定義
def predict_model():
    import tensorflow as tf
    import numpy as np
    model = tf.keras.models.load_model('/mnt/demo-vol-01/mnist_saved_model')
    new_data = np.random.rand(28, 28)
    new_data = new_data.reshape(1, 28, 28) / 255.0
    predictions = model.predict(new_data)
    predicted_class = np.argmax(predictions, axis=1)
    print("Predicted class:", predicted_class)
    with open('/mnt/demo-vol-01/predictions.txt', 'w') as f:
        f.write("Predicted class: " + str(predicted_class[0]) + "\n")

# コンテナオペレーションを作成
train_and_save_op = comp.func_to_container_op(train_and_save_model, base_image='tensorflow/tensorflow:latest')
predict_op = comp.func_to_container_op(predict_model, base_image='tensorflow/tensorflow:latest')

# パイプラインに推論タスクを追加
@dsl.pipeline(
    name='Mnist Training and Prediction Pipeline',
    description='A pipeline that trains an MNIST model, saves it to PVC and makes a prediction.'
)

def mnist_pipeline():
    # トレーニングタスク
    train_and_save_task = train_and_save_op()
    train_and_save_task.add_pvolumes({'/mnt/demo-vol-01': dsl.PipelineVolume(pvc="demo-vol-01")})

    # 推論タスク
    predict_task = predict_op()
    predict_task.add_pvolumes({'/mnt/demo-vol-01': dsl.PipelineVolume(pvc="demo-vol-01")})

    predict_task.after(train_and_save_task)

# コンパイル
kfp.compiler.Compiler().compile(mnist_pipeline, 'mnist_pipeline_and_predict.yaml')

コンパイルで生成されたYAMLを確認する。

In [ ]:
!cat mnist_pipeline_and_predict.yaml

YAMLをアップロードする。

生成されたYAMLはローカルにダウンロードして、Kubeflow UIの「Pipelines (KFP)」からアップロードする。
* ブラウザで、Kubeflow UI を開く
* Pipelines 画面を開く
* 以前にアップロードしたパイプラインを開く
* 「Upload version」からアップロードする

# パイプラインの実行

パイプラインを実行する。
* Experimentsの作成（ex-01）
* 「Create Run」で、パイプラインを実行する。

# パイプラインの作成（コンテナ）

トレーニングと推論のコードを、それぞれNGCのTensorFlowイメージでコンテナ化して利用する。

* コンテナは ACR に配置ずみ。（ServiceAccount default-editorに、ImagePullSecretが必要）
* PVC demo-vol-01 が必要

パイプラインをコンパイルするためのコード

トレーニングと推論のコードは、コンテナ化して利用する。この定義は、下記のYAMLファイルとして分離されている。
* train_and_save_model.yaml
* predict_model.yaml

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp

# コンポーネントをファイルからロード
train_and_save_op = kfp.components.load_component_from_file('train_and_save_model.yaml')
predict_op = kfp.components.load_component_from_file('predict_model.yaml')

@dsl.pipeline(
    name='MNIST Training and Prediction Pipeline',
    description='A pipeline that trains an MNIST model, saves it to PVC and makes a prediction.'
)

def mnist_pipeline():
    # トレーニングタスク
    train_and_save_task = train_and_save_op()
    train_and_save_task.add_pvolumes({'/mnt/demo-vol-01': dsl.PipelineVolume(pvc="demo-vol-01")})

    # 推論タスク
    predict_task = predict_op()
    predict_task.add_pvolumes({'/mnt/demo-vol-01': dsl.PipelineVolume(pvc="demo-vol-01")})
    predict_task.after(train_and_save_task)

# コンパイル
kfp.compiler.Compiler().compile(mnist_pipeline, 'mnist_pipeline_and_predict_ctr.yaml')

YAMLが生成されたことを確認する。

In [ ]:
!ls -l mnist_pipeline_and_predict_ctr.yaml

YAMLファイルは、Kubeflow PipelinesのUIからアップロードする。

# PVに保存されたデータの確認

PVをマウントしたPodを起動して、保存されたデータを確認する。

In [ ]:
!kubectl apply -f pod.yaml

In [ ]:
!kubectl get pod

In [ ]:
!kubectl exec demo-vol-01-pod  -- df /mnt

In [ ]:
!kubectl exec demo-vol-01-pod -- ls /mnt

In [ ]:
!kubectl exec demo-vol-01-pod  -- cat /mnt/predictions.txt